In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf

import cv2
import matplotlib.pyplot as plt
from sklearn.cross_validation import train_test_split
import random

import json

from glob import glob

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
# GPU selection (memory)
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
#os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
"""
f_dir1 = '/models/ICROS_vgg/Train_0/model.ckpt-2280.meta'
f_dir2 = '/models/ICROS_vgg/Train_0/'


saver = tf.train.import_meta_graph(f_dir1)
batch = 64

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    for op in tf.get_default_graph().get_operations():
        print(op.name)
        #test = graph.get_tensor_by_name("dropout6/dropout/keep_prob:0")
        #print(op.values())
"""

'\nf_dir1 = \'/models/ICROS_vgg/Train_0/model.ckpt-2280.meta\'\nf_dir2 = \'/models/ICROS_vgg/Train_0/\'\n\n\nsaver = tf.train.import_meta_graph(f_dir1)\nbatch = 64\n\nwith tf.Session() as sess:\n    \n    new_saver = tf.train.import_meta_graph(f_dir1)\n    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))\n    graph = tf.get_default_graph()\n    for op in tf.get_default_graph().get_operations():\n        print(op.name)\n        #test = graph.get_tensor_by_name("dropout6/dropout/keep_prob:0")\n        #print(op.values())\n'

In [4]:
# Data info.

width = 640
height = 480
channel = 3


# resize
r_w = 224
r_h = 224

total_pix = r_w * r_h * channel

In [5]:
data_txt = open('./data_split/data_dir_0801.txt', 'r')
data_json = data_txt.read()
tr_data_dir = json.loads(data_json)

In [6]:
val_data = np.zeros((len(tr_data_dir['valid']), total_pix), dtype=np.float32)
val_label = np.zeros((len(tr_data_dir['valid']), 1), dtype=np.int32)

In [8]:
for i in range(len(val_data)):
    img = cv2.imread(tr_data_dir['valid'][i])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    val_data[i,:] = img2.flatten()
    if 'Untorn' in tr_data_dir['valid'][i]:
        val_label[i] = 0
    else: val_label[i] = 1

In [9]:
test_data = np.zeros((len(tr_data_dir['test']), total_pix), dtype=np.float32)
test_label = np.zeros((len(tr_data_dir['test']), 1), dtype=np.int32)

In [10]:
for i in range(len(test_data)):
    img = cv2.imread(tr_data_dir['test'][i])
    img2 = cv2.resize(img, (r_w, r_h), interpolation=cv2.INTER_CUBIC)
    test_data[i,:] = img2.flatten()
    if 'Untorn' in tr_data_dir['test'][i]:
        test_label[i] = 0
    else: test_label[i] = 1

In [31]:
f_dir1 = '/models/ICROS_vgg/Train_9/model.ckpt-1140.meta'
f_dir2 = '/models/ICROS_vgg/Train_9/'

#f_dir1 = '/models/ICROS_re/Train_3/model.ckpt-2280.meta'
#f_dir2 = '/models/ICROS_re/Train_3/'

#ev_in = test_data
#ev_in_label = test_label

ev_in = val_data
ev_in_label = val_label

saver = tf.train.import_meta_graph(f_dir1)
batch = 64
ev_steps = int(ev_in_label.shape[0]/batch)
ev_acc = np.zeros(ev_in_label.shape[0])

with tf.Session() as sess:
    
    new_saver = tf.train.import_meta_graph(f_dir1)
    new_saver.restore(sess, tf.train.latest_checkpoint(f_dir2))
    graph = tf.get_default_graph()
    #for op in tf.get_default_graph().get_operations():
        #print(op.name)
        #print(op.values())
        
    data_in = graph.get_tensor_by_name("fifo_queue_DequeueUpTo:1")

    #feed_dict={data_in:ev_data[0:10], label:ev_label[0:10]}
    keep_prob1 = graph.get_tensor_by_name("dropout6/dropout/keep_prob:0")
    keep_prob2 = graph.get_tensor_by_name("dropout7/dropout/keep_prob:0")
    for i in range(ev_steps):
        if i < (ev_steps-1):
            feed_dict={data_in:ev_in[batch*i:batch*(i+1)], keep_prob1:1.0, keep_prob2:1.0}
            label = ev_in_label[batch*i:batch*(i+1)].reshape(batch)
        else:
            feed_dict={data_in:ev_in[batch*i:], keep_prob1:1.0, keep_prob2:1.0}
            temp = ev_in_label[batch*i:]
            label = temp.reshape(len(temp))
        
        prob = graph.get_tensor_by_name("softmax_tensor:0")
        classes = graph.get_tensor_by_name("ArgMax:0")
        
        
        result_pr = sess.run(prob, feed_dict=feed_dict)
        result_cl = sess.run(classes, feed_dict=feed_dict)
        
        print(result_pr)
        print(result_cl)
        print(label)

        acc = np.equal(result_cl, label).reshape((1,len(label)))*1
        
        print(acc)
        print(np.mean(acc))
        
        if i < (ev_steps-1):
            ev_acc[batch*i:batch*(i+1)] = acc
        else:
            ev_acc[batch*i:] = acc


INFO:tensorflow:Restoring parameters from /models/ICROS_vgg/Train_9/model.ckpt-1140
[[1.3675284e-27 1.0000000e+00]
 [1.0000000e+00 2.7239205e-21]
 [1.9281580e-18 1.0000000e+00]
 [9.9582171e-01 4.1782218e-03]
 [1.7702305e-03 9.9822980e-01]
 [8.7596722e-11 1.0000000e+00]
 [3.4284274e-22 1.0000000e+00]
 [9.9999988e-01 1.5084539e-07]
 [6.5846598e-06 9.9999344e-01]
 [1.0000000e+00 1.2939859e-10]
 [4.1054878e-03 9.9589455e-01]
 [1.8986986e-11 1.0000000e+00]
 [1.0000000e+00 6.8318429e-19]
 [1.9326965e-08 1.0000000e+00]
 [1.5581554e-03 9.9844187e-01]
 [9.9999988e-01 1.0338789e-07]
 [9.5660371e-01 4.3396354e-02]
 [1.0000000e+00 2.4833878e-12]
 [9.9999988e-01 6.1601639e-08]
 [9.9975556e-01 2.4443152e-04]
 [6.9391359e-12 1.0000000e+00]
 [0.0000000e+00 1.0000000e+00]
 [1.0000000e+00 1.7726222e-23]
 [1.9357492e-10 1.0000000e+00]
 [9.9995327e-01 4.6767334e-05]
 [1.0000000e+00 2.5691588e-34]
 [6.5515183e-06 9.9999344e-01]
 [9.9999857e-01 1.3715666e-06]
 [1.0000000e+00 3.5182172e-21]
 [4.6583294e-04 9

In [32]:
print(ev_acc[-1])
print(np.mean(ev_acc))
fails = np.where(ev_acc == 0)
print(len(fails[0]))

1.0
0.9694323144104804
7
